# Introductory Demo
This is adapted from the `gpt-2-simple` README.md from https://github.com/minimaxir/gpt-2-simple

We use a very short corpus for training so that GPUs or very large compute resources are not required. This should execute on a reasonably powerful laptop. I used a 16GB memory machine with 8 cores.

In [ ]:
import gpt_2_simple as gpt2
import os
import requests

Now we load the GPT-2 model we want to use. I've picked the `124M` one (named because it has 124 million parameters) but you can select the other ones if you want. The `355M` one is quite a bit bigger and `774M` is really big.

In [ ]:
model_name = "124M"
if not os.path.isdir(os.path.join("../models", model_name)):
	print(f"Downloading {model_name} model...")
	gpt2.download_gpt2(model_name=model_name)   # model is saved into specified directory under /models/124M/

We now need to train the model on some text. I decided for this demo to just create some text about GPT-2. It's a combination of some of my own words along with copy/paste from Wikipedia articles about GPT-2 and GPT-3.

The corpus must be at least 1024 tokens (roughly words) long. I only came up with ~500 words, so decided to just duplicate it to meet the criteria. 

In [ ]:
file_name = 'demo.txt'
my_text = """My first GPT-2 model. It must have at least 1024 tokens in it.
I'm just choosing some representative text that fits with the spirit of this demo.
In some of the other notebooks we'll use more meaningful content to showcase what GPT-2 can do!
Going to fill up the rest of the required length with content from the GPT-2 and GPT-3 pages on Wikipedia.

Generative Pre-trained Transformer 2, commonly known by its abbreviated form GPT-2, is an unsupervised transformer language model and the successor to GPT. GPT-2 was first announced in February 2019, with only limited demonstrative versions initially released to the public. The full version of GPT-2 was not immediately released out of concern over potential misuse, including applications for writing fake news. Some experts expressed skepticism that GPT-2 posed a significant threat. The Allen Institute for Artificial Intelligence responded to GPT-2 with a tool to detect "neural fake news". Other researchers, such as Jeremy Howard, warned of "the technology to totally fill Twitter, email, and the web up with reasonable-sounding, context-appropriate prose, which would drown out all other speech and be impossible to filter". In November 2019, OpenAI released the complete version of the GPT-2 language model. Several websites host interactive demonstrations of different instances of GPT-2 and other transformer models.
GPT-2's authors argue unsupervised language models to be general-purpose learners, illustrated by GPT-2 achieving state-of-the-art accuracy and perplexity on 7 of 8 zero-shot tasks (i.e. the model was not further trained on any task-specific input-output examples). The corpus it was trained on, called WebText, contains slightly over 8 million documents for a total of 40 GB of text from URLs shared in Reddit submissions with at least 3 upvotes. It avoids certain issues encoding vocabulary with word tokens by using byte pair encoding. This allows to represent any string of characters by encoding both individual characters and multiple-character tokens.

The quality of the text generated by GPT-3 is so high that it is difficult to distinguish from that written by a human, which has both benefits and risks.
Thirty-one OpenAI researchers and engineers presented the original May 28, 2020 paper introducing GPT-3. In their paper, they warned of GPT-3's potential dangers and called for research to mitigate risk.
David Chalmers, an Australian philosopher, described GPT-3 as "one of the most interesting and important AI systems ever produced."
In his July 29, 2020, review in The New York Times, Farhad Manjoo said that GPT-3—which can generate computer code and poetry, as well as prose—is not just "amazing", "spooky", and "humbling", but also "more than a little terrifying".
Because GPT-3 can "generate news articles which human evaluators have difficulty distinguishing from articles written by humans," GPT-3 has the "potential to advance both the beneficial and harmful applications of language models." In their May 28, 2020 paper, the researchers described in detail the potential "harmful effects of GPT-3" which include "misinformation, spam, phishing, abuse of legal and governmental processes, fraudulent academic essay writing and social engineering pretexting". The authors draw attention to these dangers to call for research on risk mitigation.

"""
with open(file_name, 'w') as file_handler:
    file_handler.write(my_text + '\n')
    file_handler.write(my_text) # We write it twice because my_text was too small

Now that we have a text file that the model can train on, let's train it!

On my 8 core laptop with 16GB memory the cell below takes about 5 minutes to complete 10 training steps (`steps=10`). 

For a large corpus (like the `shakespeare` text) you'll want on the order of 1000 training steps. If you don't have a GPU available, consider using a cloud computing option like AWS, GCP, Azure or similar.

CAVEAT: With the way that state is managed for tensorflow within the gpt2 module you can't re-run this next cell unless you clear the kernel in between. 

In [ ]:
sess = gpt2.start_tf_sess()
gpt2.finetune(sess,
              file_name,
              model_name=model_name,
              model_dir='../models',
              checkpoint_dir='../checkpoint',
              run_name='demo',
              steps=10)   # steps is max number of training steps

Now we have our trained model! 

Let's feed in a sample sentence (`sample`) and check out 2 generated completions, each of length 100 tokens (roughly words). Note that we'll end up with hanging sentences, since we're not guaranteed to have it end with a complete sentence.

In [ ]:
sample = "What can we do with our GPT-2 model now?"

generated = gpt2.generate(sess, run_name='demo', checkpoint_dir='../checkpoint', model_dir='../models', model_name='124M',
                          return_as_list=True, prefix=sample, nsamples=2, length=100)
print("First generated= " + generated[0])
print("\n====================\n")
print("Second generated= " + generated[1])

You can alter the sample, change the number of samples to return (`nsamples`) to get more, or increase/decrease the `length` of the returned text. 

You'll also notice that the generated text varies a lot from run to run.